In [7]:
from pydantic import BaseModel, Field
from typing import List, Literal
from langchain_deepseek.chat_models import ChatDeepSeek
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser

import os

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(usecwd=True))


BASE_URL = os.getenv("LITELLM_BASE_URL", "http://a6k2.dgx:34000/v1")
API_KEY = os.getenv("LITELLM_API_KEY", "")
MODEL_NAME = os.getenv("MODEL_NAME", "qwen3-32b")

class ProblemSpec(BaseModel):
    """Problem specification for PINN"""
    problem_type: Literal["fluid_dynamics", "heat_transfer", "structural_mechanics", "epidemiology", "other"] = Field(description="Type of physical problem")
    confidence: float = Field(ge=0.0, le=1.0, description="Confidence in classification from 0 to 1")
    reasoning: str = Field(description="Explanation for the chosen problem type")

parser = PydanticOutputParser(pydantic_object=ProblemSpec)

prompt = ChatPromptTemplate(messages=[
    ("system", """You are an expert in Physics-Informed Neural Networks (PINN). 
Analyze the problem description and determine its type.
{format_instructions}
result_output = {schema}

Return ONLY valid JSON in this exact format."""),
    
    ("human", "PROBLEM DESCRIPTION: {query}")
],
partial_variables = {"format_instructions": parser.get_format_instructions()})

# prompt = ChatPromptTemplate.from_messages([
#     ("system", """You are an expert in Physics-Informed Neural Networks (PINN). 
# Analyze the problem description and determine its type.
# {format_instructions}
# result_output = {schema}

# Return ONLY valid JSON in this exact format."""),
    
#     ("human", "PROBLEM DESCRIPTION: {query}")
# ])

# prompt.partial_variables = {"format_instructions": parser.get_format_instructions()}    

llm = ChatDeepSeek(
    api_base=BASE_URL,
    base_url=BASE_URL,
    api_key=API_KEY,
    model=MODEL_NAME,
    streaming=False,
    timeout=120
)

chain = prompt | llm | parser

out: ProblemSpec =  await chain.ainvoke({"query":"We need to solve a problem of heat conduction in a rod", "schema": ProblemSpec.model_json_schema()})

print(out)

problem_type='heat_transfer' confidence=0.95 reasoning="The problem involves heat conduction in a rod, which is governed by the heat equation—a classical partial differential equation in the domain of heat transfer. This directly aligns with the 'heat_transfer' category as it deals with temperature distribution and thermal energy propagation."
